In [2]:
import sys
print(sys.executable)


c:\Users\ruchi\anaconda3\python.exe


In [3]:
import pandas as pd
from pathlib import Path
import gdown, zipfile, os

In [4]:
#https://drive.google.com/file/d/1U9P0YmhpPtaAuK9ENhObxLwy7w-Q6ggb/view?usp=drive_link

# --- Replace with your ZIP file ID ---
zip_id = "1U9P0YmhpPtaAuK9ENhObxLwy7w-Q6ggb"

# --- Local paths ---
zip_path = "Exploring_Nutrition_with_data_science/usda_data.zip"
extract_dir = "Exploring_Nutrition_with_data_science"

os.makedirs(extract_dir, exist_ok=True)

# --- Download only if not already present ---
if not os.path.exists(zip_path):
    print("⬇️ Downloading ZIP from Google Drive...")
    gdown.download(f"https://drive.google.com/uc?id={zip_id}", zip_path, quiet=False)
else:
    print("✅ ZIP already exists, skipping download.")

# --- Extract ---
print("📦 Extracting files...")
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_dir)

# --- Load CSVs ---
food = pd.read_csv(os.path.join(extract_dir, "food.csv"))
food_cat = pd.read_csv(os.path.join(extract_dir, "food_category.csv"))
food_nutrient = pd.read_csv(os.path.join(extract_dir, "food_nutrient.csv"))
nutrient = pd.read_csv(os.path.join(extract_dir, "nutrient.csv"))

print("✅ Data loaded successfully:")
print("  food:", food.shape)
print("  food_category:", food_cat.shape)
print("  food_nutrient:", food_nutrient.shape)
print("  nutrient:", nutrient.shape)


⬇️ Downloading ZIP from Google Drive...


Downloading...
From (original): https://drive.google.com/uc?id=1U9P0YmhpPtaAuK9ENhObxLwy7w-Q6ggb
From (redirected): https://drive.google.com/uc?id=1U9P0YmhpPtaAuK9ENhObxLwy7w-Q6ggb&confirm=t&uuid=296a9acc-6b80-4569-b069-086369b67a92
To: c:\Users\ruchi\Documents\Python_ironhack\extension\Exploring_Nutrition_with_data_science\Exploring_Nutrition_with_data_science\usda_data.zip
100%|██████████| 278M/278M [00:21<00:00, 12.6MB/s] 


📦 Extracting files...


C:\Users\ruchi\AppData\Local\Temp\ipykernel_12412\1658093294.py:27: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  food_nutrient = pd.read_csv(os.path.join(extract_dir, "food_nutrient.csv"))


✅ Data loaded successfully:
  food: (2064912, 5)
  food_category: (28, 3)
  food_nutrient: (26805037, 13)
  nutrient: (477, 5)


In [5]:
display(food.head(3))
display(food_nutrient.head(3))
display(nutrient.head(3))
display(food_cat.head(3))

,fdc_id,data_type,description,food_category_id,publication_date
0,1105904,branded_food,WESSON Vegetable Oil 1 GAL,Oils Edible,2020-11-13
1,1105905,branded_food,SWANSON BROTH BEEF,Herbs/Spices/Extracts,2020-11-13
2,1105906,branded_food,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,Prepared Soups,2020-11-13


,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,percent_daily_value
0,13706927,1105904,1257,0.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13706930,1105904,1293,53.33,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,13706926,1105904,1253,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0


,id,code,description
0,1,100,Dairy and Egg Products
1,2,200,Spices and Herbs
2,3,300,Baby Foods


In [18]:
  # show first 50 unique values
print(food["data_type"].value_counts())


data_type
branded_food                1977398
sub_sample_food               62022
sr_legacy_food                 7793
market_acquistion              7215
survey_fndds_food              5432
sample_food                    3717
agricultural_acquisition        810
foundation_food                 411
experimental_food               114
Name: count, dtype: int64


In [19]:
# 2) Filter only Foundation + SR Legacy foods
food = food[food["data_type"].isin(["foundation_food", "sr_legacy_food"])]
print("\nAfter filtering Foundation + SR Legacy:", food.shape)
display(food.head(5))


After filtering Foundation + SR Legacy: (8204, 5)


,fdc_id,data_type,description,food_category_id,publication_date
32885,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01
32886,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",18,2019-04-01
32887,167514,sr_legacy_food,"Kraft Foods, Shake N Bake Original Recipe, Coa...",18,2019-04-01
32888,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",18,2019-04-01
32889,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",18,2019-04-01


In [24]:
# ------------------------------------------------------------
# 3) Merge food with category descriptions
# ------------------------------------------------------------

# Drop duplicate category/code columns from food
food = food.drop(columns=[col for col in food.columns if "Category" in col or "code" in col], errors="ignore")

# Prepare food_category table
food_cat_small = food_cat[["id", "code", "description"]].rename(
    columns={
        "id": "food_category_id",
        "code": "CategoryCode",
        "description": "Category"
    }
)

# 🔹 Ensure both sides are the same type before merge
food["food_category_id"] = food["food_category_id"].astype(str)
food_cat_small["food_category_id"] = food_cat_small["food_category_id"].astype(str)

# Merge
food = food.merge(food_cat_small, on="food_category_id", how="left")

print("\nAfter merging categories:", food.shape)
display(food.head(5))



After merging categories: (8204, 7)


,fdc_id,data_type,description,food_category_id,publication_date,CategoryCode,Category
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1800,Baked Products
1,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",18,2019-04-01,1800,Baked Products
2,167514,sr_legacy_food,"Kraft Foods, Shake N Bake Original Recipe, Coa...",18,2019-04-01,1800,Baked Products
3,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",18,2019-04-01,1800,Baked Products
4,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",18,2019-04-01,1800,Baked Products


In [21]:
print(nutrient.columns.tolist())
display(nutrient)


['id', 'name', 'unit_name', 'nutrient_nbr', 'rank']


,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0
3,1002,Nitrogen,G,202.0,500.0
4,1003,Protein,G,203.0,600.0
...,...,...,...,...,...
472,2064,Oligosaccharides,MG,NaN,2250.0
473,2065,Low Molecular Weight Dietary Fiber (LMWDF),G,293.4,1306.0
474,2068,Vitamin E,MG,959.0,7810.0
475,2067,Vitamin A,UG,960.0,7430.0


In [22]:
# 4) Prepare nutrient mapping (force kcal, not kJ)
# ------------------------------------------------------------
canonical_map = {
    "Energy (Atwater General Factors)": "Calories (kcal)", 
    "Energy":"Calories (kcal)",  # kcal
    "Protein": "Protein (g)",
    "Carbohydrate, by difference": "Carbs (g)",
    "Total lipid (fat)": "Fat (g)",
    "Fiber, total dietary": "Fiber (g)"
}

# Merge nutrients with their names + units
fn = food_nutrient.merge(
    nutrient[["id", "name", "unit_name"]],
    left_on="nutrient_id", right_on="id", how="left"
)
# 🔑 Force Energy selection: keep only kcal rows when multiple
fn = fn[~((fn["name"].str.contains("Energy", case=False, na=False)) & (fn["unit_name"] == "kJ"))]
fn["canonical"] = fn["name"].map(canonical_map)
print("\nAfter merging nutrients:", fn.shape)
display(fn.head(5))


After merging nutrients: (26797076, 17)


,id_x,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,percent_daily_value,id_y,name,unit_name,canonical
0,13706927,1105904,1257,0.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1257,"Fatty acids, total trans",G,NaN
1,13706930,1105904,1293,53.33,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1293,"Fatty acids, total polyunsaturated",G,NaN
2,13706926,1105904,1253,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1253,Cholesterol,MG,NaN
3,13706921,1105904,1092,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1092,"Potassium, K",MG,NaN
4,13706916,1105904,1008,867.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1008,Energy,KCAL,Calories (kcal)


In [23]:
# --- Hybrid Sugar Handling ---
# SR Legacy → already has "Total Sugars"
# Foundation → sum of individual sugars

# 1) Extract total sugars if present
total_sugars = (
    fn[fn["name"].str.contains("Total Sugars", case=False)]
    .groupby("fdc_id")["amount"]
    .first()  # take the single value if multiple rows
    .reset_index()
    .rename(columns={"amount": "Sugar (g)"})
)

# 2) Aggregate simple sugars for foods without "Total Sugars"
sugar_names = ["Glucose", "Fructose", "Sucrose", "Lactose", "Maltose"]
sugar_parts = (
    fn[fn["name"].isin(sugar_names)]
    .groupby("fdc_id")["amount"]
    .sum()
    .reset_index()
    .rename(columns={"amount": "Sugar (g)"})
)

# 3) Combine both — prefer "Total Sugars" if available
sugars = pd.concat([total_sugars, sugar_parts]) \
    .groupby("fdc_id", as_index=False)["Sugar (g)"].first()

print("\nFinal sugars table:", sugars.shape)
display(sugars.head(10))

# --- Merge with pivot nutrients ---
pivot = fn[fn["canonical"].notna()].pivot_table(
    index="fdc_id", 
    columns="canonical", 
    values="amount", 
    aggfunc="first"
).reset_index()

pivot = pivot.merge(sugars, on="fdc_id", how="left")

print("\nAfter pivot + sugars:", pivot.shape)
display(pivot.head(10))



Final sugars table: (1783523, 2)


,fdc_id,Sugar (g)
0,167512,5.88
1,167513,21.34
2,167516,4.30
3,167517,4.41
4,167518,4.50
5,167519,5.04
6,167520,18.13
7,167521,26.31
8,167522,22.02
9,167524,13.10



After pivot + sugars: (1889007, 7)


,fdc_id,Calories (kcal),Carbs (g),Fat (g),Fiber (g),Protein (g),Sugar (g)
0,167512,307.0,41.18,13.24,1.2,5.88,5.88
1,167513,330.0,53.42,11.27,1.4,4.34,21.34
2,167514,377.0,79.80,3.70,NaN,6.10,NaN
3,167515,232.0,46.00,1.80,NaN,8.00,NaN
4,167516,273.0,41.05,9.22,2.2,6.58,4.30
5,167517,309.0,48.39,9.49,2.6,7.42,4.41
6,167518,289.0,44.16,9.40,2.4,6.92,4.50
7,167519,298.0,45.41,9.91,2.4,6.71,5.04
8,167520,501.0,64.30,24.83,1.9,5.10,18.13
9,167521,484.0,64.48,22.42,2.7,6.08,26.31


In [25]:
final = food.merge(pivot, on="fdc_id", how="left")

# Rename nicely
final = final.rename(columns={
    "description": "Food",
    "brand_owner": "Brand Owner",
    "brand_name": "Brand Name",
    "gtin_upc": "UPC",
    "serving_size": "Serving Size",
    "serving_size_unit": "Serving Unit"
})
print("Final dataset:", final.shape)
display(final)


Final dataset: (8204, 13)


,fdc_id,data_type,Food,food_category_id,publication_date,CategoryCode,Category,Calories (kcal),Carbs (g),Fat (g),Fiber (g),Protein (g),Sugar (g)
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1800,Baked Products,307.0,41.18,13.24,1.2,5.880000,5.88000
1,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",18,2019-04-01,1800,Baked Products,330.0,53.42,11.27,1.4,4.340000,21.34000
2,167514,sr_legacy_food,"Kraft Foods, Shake N Bake Original Recipe, Coa...",18,2019-04-01,1800,Baked Products,377.0,79.80,3.70,NaN,6.100000,NaN
3,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",18,2019-04-01,1800,Baked Products,232.0,46.00,1.80,NaN,8.000000,NaN
4,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",18,2019-04-01,1800,Baked Products,273.0,41.05,9.22,2.2,6.580000,4.30000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8199,2727585,foundation_food,"Green onion, (scallion), bulb and greens, root...",11,2025-04-24,1100,Vegetables and Vegetable Products,NaN,NaN,NaN,NaN,0.668750,2.61800
8200,2727586,foundation_food,"Shallots, bulb, peeled, root removed, raw",11,2025-04-24,1100,Vegetables and Vegetable Products,NaN,NaN,NaN,NaN,1.377500,4.35100
8201,2727587,foundation_food,"Juice, prune, shelf-stable",9,2025-04-24,900,Fruits and Fruit Juices,NaN,NaN,NaN,NaN,0.422625,14.81250
8202,2727588,foundation_food,"Juice, pomegranate, from concentrate, shelf-st...",9,2025-04-24,900,Fruits and Fruit Juices,NaN,NaN,NaN,NaN,0.000000,13.27675


In [26]:
# Count NaN values in selected nutrient columns
cols_to_check = ["Calories (kcal)", "Protein (g)", "Carbs (g)", "Fat (g)", "Fiber (g)", "Sugar (g)"]

nan_counts = final[cols_to_check].isna().sum()
print("NaN counts per column:\n", nan_counts)


NaN counts per column:
 Calories (kcal)      58
Protein (g)          12
Carbs (g)            59
Fat (g)              23
Fiber (g)           766
Sugar (g)          1987
dtype: int64


In [27]:
final[["Fiber (g)", "Sugar (g)"]] = final[["Fiber (g)", "Sugar (g)"]].fillna(0)
# Select only the useful columns
cols_to_keep = [
    "Food", "Category", "Calories (kcal)", 
    "Protein (g)", "Carbs (g)", "Fat (g)", "Fiber (g)", "Sugar (g)"
]

# Subset and drop duplicates
final = final[cols_to_keep].drop_duplicates().reset_index(drop=True)

print("Final dataset:", final.shape)
display(final)



Final dataset: (8157, 8)


,Food,Category,Calories (kcal),Protein (g),Carbs (g),Fat (g),Fiber (g),Sugar (g)
0,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",Baked Products,307.0,5.880000,41.18,13.24,1.2,5.88000
1,"Pillsbury, Cinnamon Rolls with Icing, refriger...",Baked Products,330.0,4.340000,53.42,11.27,1.4,21.34000
2,"Kraft Foods, Shake N Bake Original Recipe, Coa...",Baked Products,377.0,6.100000,79.80,3.70,0.0,0.00000
3,"George Weston Bakeries, Thomas English Muffins",Baked Products,232.0,8.000000,46.00,1.80,0.0,0.00000
4,"Waffles, buttermilk, frozen, ready-to-heat",Baked Products,273.0,6.580000,41.05,9.22,2.2,4.30000
...,...,...,...,...,...,...,...,...
8152,"Green onion, (scallion), bulb and greens, root...",Vegetables and Vegetable Products,NaN,0.668750,NaN,NaN,0.0,2.61800
8153,"Shallots, bulb, peeled, root removed, raw",Vegetables and Vegetable Products,NaN,1.377500,NaN,NaN,0.0,4.35100
8154,"Juice, prune, shelf-stable",Fruits and Fruit Juices,NaN,0.422625,NaN,NaN,0.0,14.81250
8155,"Juice, pomegranate, from concentrate, shelf-st...",Fruits and Fruit Juices,NaN,0.000000,NaN,NaN,0.0,13.27675


In [28]:
# Save to CSV
final.to_csv("foundation_sr.csv", index=False)
print("✅ Saved as foundation_sr.csv")

✅ Saved as foundation_sr.csv
